In [129]:
from pyspark import SparkContext
from itertools import combinations
import sys
import pandas as pd
import numpy as np
import math
from pandas import pivot_table 


t=0.05
userid=73
numoftest=600

In [130]:
### find candidate pairs from bands
# x is row number, i is range from 0 to 19
def h(x,i):
    return (3*x+13*i)%100

#get first 3 movies
def sort(x):
    test=x[1]
    test.sort()
    test=[test[p][1] for p in range(len(test)) if p<numoftest]
    return (int(x[0]),test)

def Cosine(user1,user2,rate_df):
    user1=rate_df.loc[user1,:]
    user2=rate_df.loc[user2,:]
    cos=sum(map(lambda Xi, Yi: Xi * Yi, user1, user2))/math.sqrt(sum([a**2 for a in user1])*sum([b**2 for b in user2]))
    return cos



def candidates(lines):
    li=[]
    for j in range(0,20):
        li+=lines.map(lambda x:x.split(",",1)).flatMapValues(
            lambda s:s.split(",")).map(
            lambda x:((x[0],int(j/4)),h(eval(x[1]),j))).reduceByKey(min).collect()

    data = sc.parallelize(li, 4)
    li=data.groupByKey().map(lambda x:(x[0],list(x[1]))).map(lambda x:((x[0][1],str(x[1])),x[0][0])).groupByKey().map(lambda x:(x[0],list(x[1]),len(list(x[1])))).filter(lambda x:x[2]>1).collect()

    candidate=[]
    
    for i in li:
        if(str(userid) in i[1]):
            test=[int(j) for j in i[1] if int(j)!=userid]
            candidate+=test
    candidate=list(set(candidate))
    candidate=[(userid,j) for j in candidate]
    
    return candidate


### find candidate similarity on whole dataset
def recheck(da,candidate,rate_df):
    data={}
    recommend={str(userid):[]}
    #da=lines.map(lambda x:x.split(",",1)).map(lambda x:(x[0],eval(x[1]))).collect()
    for i in da:
        data[i[0]]=i[1]
        #recommend[i[0]]=[]
    for j in candidate:
        cos=Cosine(j[0],j[1],rate_df)
        if(cos>t):
            recommend[str(j[0])].append(((round(cos,6),int(j[1]),data.get(str(j[1])))))
            
    return recommend


def getwatch_history(rating):
    watch_history=rating.groupby(["userId"], as_index=False)["movieId"].apply(list)
    wh={}
    start=1
    for i in watch_history:
        wh[start]=tuple(i)
        start+=1
    return wh

def remove_watched(wh,recommend):
    new_recommend={}
    for i in recommend:
        new_recommend[i]=[]
        content=recommend[i]
        watched=wh.get(int(i))
        for j in content:
            unwatched=tuple(set(j[2])-set(watched))
            if(len(unwatched)>0):
                new_recommend[i]+=[(j[0],j[1],unwatched)]
    return new_recommend

def rec(recommend):
    re=[]
    for i in recommend:
        te=recommend.get(i)
        te.sort(reverse=True)
        removie=[]
        for j in range(len(te)):
            if(j<5):
                if(type(te[j][2])==int):
                    removie.append(((i,te[j][2]),1))
                else:
                    removie+=[((i,p),1) for p in te[j][2]]
        re+=removie
    return re


def remove_test(rating, wh, userid, test):
    
    # remove from rating and store test as test
    user_movies=rating[rating["userId"]==userid]
    rating=rating[rating["userId"]!=userid]
    train=user_movies.sample(n=(len(user_movies)-test),random_state=1)
    rating=rating.append(train).reset_index()
    
    test_index=set(user_movies.index)-set(train.index)
    test_movies=user_movies.loc[test_index,:]
    
    # remove from wh
    wh[userid]=tuple(set(wh[userid])-set(train.index))

    return test_movies, rating
    
    

In [131]:
# input



rating=pd.read_csv("ratings_small.csv")

wh=getwatch_history(rating)
# remove test datasc
test_movies,rating=remove_test(rating, wh, userid, numoftest)
rate_df = pivot_table(rating, values='rating', index=['userId'],
                    columns=['movieId'])
row_average=list(rate_df.apply(np.mean,axis="columns"))
rate_df=rate_df.sub(row_average,axis='index')#.fillna(0)

#rate_df
 
# prepare for candidates
#positive_review=rate_df[rate_df>0]
lines=[]
for i in rate_df.index:
    temp=[i]+list(rate_df.loc[i,:].dropna().index)
    lines+=[str(temp).replace("[","").replace("]","")]
    
    
sc = SparkContext.getOrCreate()
lines = sc.parallelize(lines)
totalnum=len(lines.collect())
candidate=candidates(lines)

ls=[]
for i in wh.keys():
    ls+=[(str(i),wh.get(i))]

rate_df=rate_df.fillna(0)
recommend=recheck(ls,candidate,rate_df)
new_recommend=remove_watched(wh,recommend)
re=rec(new_recommend)

rdd=sc.parallelize(re,3)
output=rdd.groupByKey().map(
        lambda x:(x[0][0],(int(totalnum-sum(x[1])),x[0][1]))).groupByKey().map(
        lambda x:(x[0],list(x[1]))).map(
        lambda x: sort(x)).sortByKey(True).map(lambda x:("U "+str(x[0]),x[1])).collect()

rdd_real=sc.parallelize(test_movies["movieId"])
rdd_recommend=sc.parallelize(output[0][1])

print(rdd_real.intersection(rdd_recommend).collect())

[]
